# TEST

In [ ]:
from scripts.forecast import GeneralRegression
import matplotlib.pyplot as plt
import numpy as np

n = opbrengsten.size
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title('Opbrengsten voorbije 5 jaar')
ax.set_xlabel('kwartaal')
ax.set_ylabel('opbrengst (€)')
ax2 = ax.secondary_xaxis('top')
ax2.set_xticks(range(n))
ax2.set_xticklabels(['Q{}'.format(j % 4 + 1) for j in range(n)])
ax.set_xticks(range(n))
ax.plot(opbrengsten, label='gegeven', color='C0', marker='o')
for i in range(0, n, 4):
    ax.axvline(i, color='gray', linewidth=0.5)
ax.legend()



def naive(y: np.array):
    if y.size > 1:
        return y[-1]
    return np.NaN

def average(y: np.array):
    if y.size < 1:
        return np.NaN
    return y.mean()

def moving_average(y: np.array, m=4):
    if y.size < m:
        return np.NaN
    return np.mean(y[-m:])

def bereken_gewichten(y: np.array, m: int):
    n = y.size
    if n < 2 * m:
        return np.NaN
    M = y[-(m + 1):-1]
    for i in range(1, m):
        M = np.vstack([M, y[-(m + i + 1):-(i + 1)]])
    v = np.flip(y[-m:])
    return np.linalg.solve(M, v)

def linear_combination(y: np.array, m=4) -> np.ndarray:
    n = y.size
    # check op minstens 2*m gegevens
    if n < 2 * m:
        return np.NaN
    # bereken de gewichten
    a = bereken_gewichten(y, m)
    # bereken de voorspelde waarde en geef de voorspelde waarde terug
    return np.sum(y[-m:] * a)

def create_trend_model(y):
    # we bouwen een lineaire regressie model
    X = np.arange(0, y.size)
    model = GeneralRegression()
    model.fit(X, y)
    # we geven een functie terug!
    return lambda x: model.predict([[x]])

fig, ax = plt.subplots(figsize=(10, 5))
lags, acfs, _, _ = ax.acorr(opbrengsten)
ax.set_xticks(range(-10, 11))
ax.set_xlabel('offset')
ax.set_ylabel('correlatie')
ax.set_title('Auto-correlation opbrengsten')

def find_period(y: np.array, maxlags=10, top_n=1) -> int:
    # autocorrelatie aan beide zijden
    acfs = np.correlate(y, y, mode='full') / np.sum(y ** 2)
    # midden bepalen
    middle = acfs.size // 2
    # omgekeerde argsort vanaf (midden + 1) tot maxlags + top selectie
    return (np.argsort(-1 * acfs[middle + 1: middle + maxlags]) + 1)[:top_n]

m = find_period(opbrengsten)


from statsmodels.tsa.seasonal import seasonal_decompose

sd_model = seasonal_decompose(opbrengsten, model='multiplicative', period=4)
# de verschillende componenten van het model zitten in trend, seasonal en resid
sd_model.trend
sd_model.seasonal
sd_model.resid


def seasonal_decomposition_forecast(reg_model: GeneralRegression, sd_model, start, end, method='additive', m=None):
    if not m:
        m = find_period(sd_model.observed)[0]

    # seizoenen voldoende herhalen tot voorbij 'end'
    seasonal = np.tile(sd_model.seasonal[0:m], end // m + 1)
    if method.startswith('m'):
        return reg_model.predict(np.arange(start, end)) * seasonal[start:end]
    else:
        return reg_model.predict(np.arange(start, end)) + seasonal[start:end]




# bereken het voortschrijdend gemiddelde langs 2 kanten met een venster van 3 periodes
rolling_forward = trend.rolling(window=12).mean()
rolling_backward = trend.iloc[::-1].rolling(window=12).mean().iloc[::-1]
rolling_twosided = (rolling_forward + rolling_backward) / 2


def plot_seasonal_decompositon(sd_model, title: str):
    fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(8, 8))

    axes[0].plot(sd_model.observed, 'o-', label='observed')
    axes[0].set_ylabel('observed')
    axes[0].set_title(title)
    axes[0].legend()

    axes[1].plot(sd_model.trend, 'o-', color='orange', label='trend')
    axes[1].set_ylabel('trend')
    axes[1].legend()

    axes[2].plot(sd_model.seasonal, 'o-', color='green', label='seasonal')
    axes[2].set_ylabel('season')
    axes[2].legend()

    axes[3].scatter(range(sd_model.nobs[0]), sd_model.resid, color='darkgrey', label='noise')
    axes[3].set_ylabel('residue')
    axes[3].set_xlabel('kwartaal')
    axes[3].legend()

plot_seasonal_decompositon(sd_model, 'Multiplicative')